In [1]:
a = {"a", "b"}
str(a)

"{'b', 'a'}"

In [1]:
import os
from chat import OpenAIChatModule
from dotenv import load_dotenv
load_dotenv('../.env')


chat = OpenAIChatModule(
    api_key_env_name="OPENAI_API_KEY",
    model_name="gpt-3.5-turbo-0613",
)

message = "こんにちは"
response = chat.run(message)
print(response)

KeyError: 'formatters'

In [3]:
response

CompletionResults(message={'role': 'assistant', 'content': 'こんにちは！何かお手伝いできますか？'}, usage=Usage(prompt_tokens=8, completion_tokens=14), prompt=[{'role': 'user', 'content': 'こんにちは'}])

In [5]:
# For streaming response

message = "やあ"
response = chat(message, stream=True)

prev_response = ""
for chunk in response:
    if isinstance(chunk, str):
        delta = chunk.replace(prev_response, "")
        print(delta, end="")
        prev_response = chunk
    else:
        break
print("\n===\n")
print(chunk)

Message: [{'role': 'user', 'content': 'やあ'}]
こんにちは！何かお手伝いできますか？
===

message={'role': 'assistant', 'content': 'こんにちは！何かお手伝いできますか？'} usage=Usage(prompt_tokens=3, completion_tokens=15) prompt=[{'role': 'user', 'content': 'やあ'}]


In [12]:
# For async response
import asyncio

async def call_gpt(message): 
    response = ""  
    print("Generating response... for " + message)
    response = await chat.arun(message)
    print("Response generated!")
    if response != "":
        print(response)

asyncio.create_task(call_gpt("猫の名前を決めてください"))
asyncio.create_task(call_gpt("犬の名前を決めてください"))

print("This is printed before the response is generated!")

This is printed before the response is generated!


Generating response... for 猫の名前を決めてください
Generating response... for 犬の名前を決めてください
Response generated!
message={'role': 'assistant', 'content': 'はい、どのような名前がお好みですか？性別や特徴など教えていただけると、より具体的な名前を提案できます。'} usage=Usage(prompt_tokens=19, completion_tokens=51) prompt=[{'role': 'user', 'content': '猫の名前を決めてください'}]
Response generated!
message={'role': 'assistant', 'content': 'もちろんです！以下にいくつかの犬の名前の提案をさせていただきます。\n\n1. チャーリー\n2. マックス\n3. ベイリー\n4. ルーシー\n5. ロッキー\n6. レオ\n7. ルナ\n8. ココ\n9. ボクサー\n10. ダイアナ\n\nどれか気に入った名前はありますか？もしご希望の名前がなければ、もう少し情報を教えていただければ、より適切な名前を提案できるかもしれません。'} usage=Usage(prompt_tokens=19, completion_tokens=175) prompt=[{'role': 'user', 'content': '犬の名前を決めてください'}]


In [3]:
# Conversation memory

from memory import JSONConversationMemory

chat = OpenAIChatModule(
    api_key_env_name="OPENAI_API_KEY",
    model_name="gpt-3.5-turbo-0613",
    conversation_memory=JSONConversationMemory("conversation.json")
)

while True:
    message = input("You: ")
    response = chat.run(message)
    print(response.message)
    
    if message == "bye":
        break

with open("conversation.json") as f:
    print(f.read())

Total tokens: 0
Message: {'role': 'user', 'content': 'やあ'}
{'role': 'assistant', 'content': 'こんにちは！何かお手伝いできますか？'}
Total tokens: 0
Message: {'role': 'user', 'content': '特にないよ'}
Total tokens: 6
Message: {'role': 'assistant', 'content': 'こんにちは！何かお手伝いできますか？'}
Total tokens: 21
Message: {'role': 'user', 'content': 'やあ'}
{'role': 'assistant', 'content': 'そうですか、もし何か質問や相談があれば遠慮なくお知らせください。お手伝いできることがあれば喜んでお応えします！'}
Total tokens: 0
Message: {'role': 'user', 'content': 'わかった'}
Total tokens: 5
Message: {'role': 'assistant', 'content': 'そうですか、もし何か質問や相談があれば遠慮なくお知らせください。お手伝いできることがあれば喜んでお応えします！'}
Total tokens: 64
Message: {'role': 'user', 'content': '特にないよ'}
Total tokens: 70
Message: {'role': 'assistant', 'content': 'こんにちは！何かお手伝いできますか？'}
Total tokens: 85
Message: {'role': 'user', 'content': 'やあ'}
{'role': 'assistant', 'content': '分かりました。もし何か困ったことや質問が生じたら、いつでもお知らせください。お手伝いできることがあれば喜んでお応えします。それでは、どうぞ良い一日をお過ごしください！'}
Total tokens: 0
Message: {'role': 'user', 'content': 'はーい'}
Total tokens: 4
Message: {'rol

In [2]:
# Adjusting context length

from memory import JSONConversationMemory
from truncate import OldConversationTruncationModule

chat = OpenAIChatModule(
    api_key_env_name="OPENAI_API_KEY",
    model_name="gpt-3.5-turbo-0613",
    conversation_memory=JSONConversationMemory("conversation.json"),
    conversation_length_adjuster=OldConversationTruncationModule(
        model_name="gpt-3.5-turbo-0613",
        context_length=400
    )
)

while True:
    message = input("You: ")
    response = chat.run(message)
    print(response.message)
    
    if message == "bye":
        break

Message: [{'role': 'user', 'content': ''}, {'role': 'assistant', 'content': "I'm sorry, I cannot continue the text for you as I am an AI language model and I do not have access to personal information or the ability to generate personal experiences."}, {'role': 'user', 'content': 'おはよう'}, {'role': 'assistant', 'content': 'おはようございます！朝ですね。今日はどんな一日になるか楽しみですね。何か特別な予定や目標はありますか？私はいつでもお手伝いできるので、何かお困りのことがあれば教えてください！'}, {'role': 'user', 'content': ''}, {'role': 'assistant', 'content': 'おはようございます！特別な予定や目標はありませんが、今日は仕事や勉強に集中して過ごしたいと思っています。何かアドバイスやモチベーションをくれる言葉はありますか？'}, {'role': 'user', 'content': ''}]
messages: [{'role': 'user', 'content': ''}, {'role': 'assistant', 'content': "I'm sorry, I cannot continue the text for you as I am an AI language model and I do not have access to personal information or the ability to generate personal experiences."}, {'role': 'user', 'content': 'おはよう'}, {'role': 'assistant', 'content': 'おはようございます！朝ですね。今日はどんな一日になるか楽しみですね。何か特別な予定や目標はありますか？私はいつでもお手伝いできるので、何かお困りのことが